# Extraction of clip of interest from original Ego4D videos based on Top 50 queries and ground truth

### **Setup AWS Environment**

This section sets up the environment variables and installs the AWS CLI required for interacting with the Ego4D dataset hosted on Amazon S3.

1. **Environment Variables:**
   - The `AWS_ACCESS_KEY_ID` and `AWS_SECRET_ACCESS_KEY` are set as environment variables to authenticate access to the S3 bucket containing the Ego4D dataset.

2. **Install AWS CLI:**
   - The latest version of the AWS CLI is downloaded and installed to enable direct interaction with Amazon S3.
   - After installation, the AWS CLI is configured with the credentials provided via the `aws configure` command.

3. **Clean Up:**
   - The temporary ZIP file used for downloading the AWS CLI (`awscliv2.zip`) is removed to keep the environment clean.

In [1]:
import os
os.environ['AWS_ACCESS_KEY_ID'] = "< insert secret key here >"
os.environ['AWS_SECRET_ACCESS_KEY'] = "< insert secret key here >"

In [ ]:
# Download the AWS and Ego4D CLIs
%%bash

# Set up the AWS CLI
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
unzip -o awscliv2.zip >/dev/null
sudo ./aws/install >/dev/null 2>&1
aws configure set aws_access_key_id "$AWS_ACCESS_KEY_ID" && aws configure set aws_secret_access_key "$AWS_SECRET_ACCESS_KEY"
rm "awscliv2.zip"

### **Download Manifest File**

This command downloads the `manifest.csv` file from the Ego4D dataset hosted on Amazon S3.

1. **Purpose:**
   - The `manifest.csv` file contains metadata that maps `video_uid` to their respective S3 paths. This is essential for locating and downloading specific videos or other dataset components.

2. **Command Breakdown:**
   - `aws s3 cp`: Copies a file from the specified S3 bucket to the local machine.
   - `s3://ego4d-consortium-sharing/public/v1/full_scale/manifest.csv`: Path to the `manifest.csv` file in the Ego4D S3 bucket.
   - `/content/manifest.csv`: Destination path where the `manifest.csv` file is saved in the Colab environment.

3. **Output:**
   - Upon successful execution, the `manifest.csv` file is saved in the `/content` directory for further use.

In [ ]:
!aws s3 cp s3://ego4d-consortium-sharing/public/v1/full_scale/manifest.csv /content/manifest.csv

In [ ]:
# Install required tools
!sudo apt update && sudo apt install -y ffmpeg
!pip install boto3
!pip install av

# Import required libraries
import os
import json
import av

# Set paths
top_queries_file = "/content/top_queries.json"  # Path to your top_queries.json file
manifest_path = "/content/manifest.csv"  # Path to the downloaded manifest.csv
output_dir = "/content/extracted_clips"  # Directory to save extracted clips
os.makedirs(output_dir, exist_ok=True)

### **Process Top Queries and Extract Relevant Clips**

This section processes the `top_queries.json` file to download and extract the relevant video clips based on the ground truth time intervals.

#### **1. Load Top Queries**
- The `top_queries.json` file is loaded into memory.
- Each query contains metadata such as `video_id`, `ground_truth` (start and end times), and additional attributes.

#### **2. Create Mapping of `video_id` to S3 Path**
- The `manifest.csv` file is parsed to create a dictionary (`video_mapping`) that maps `video_id` to the corresponding S3 path.
- Each line in the `manifest.csv` contains:
  - Column 1: `video_id`
  - Column 2: S3 path to the video file.

#### **3. Download and Extract Clips**
- For each query in `top_queries`:
  - Retrieve the `video_id` and ground truth time intervals (`start_time` and `end_time`).
  - If the `video_id` is not found in `video_mapping`, skip the query with a warning message.
  - Download the full video using the S3 path and save it locally.

#### **4. Extract Relevant Clip**
- Use **FFmpeg** to extract the specified segment from the downloaded video.
- The extracted clip is saved with a filename format that includes:
  - `video_id`
  - `start_time`
  - `end_time`

#### **5. Clean Up**
- After extracting the clip, the full video file is deleted to save storage space in the Colab environment.

#### **6. Final Output**
- All extracted clips are saved in the `output_dir` directory.
- A summary message is printed to indicate the completion of the process.

In [5]:
# Load top queries
with open(top_queries_file, 'r') as f:
    top_queries = json.load(f)

# Step 1: Create a mapping of video_id to S3 path
video_mapping = {}
with open(manifest_path, 'r') as f:
    for line in f:
        parts = line.strip().split(',')
        if len(parts) >= 2:
            video_id = parts[0]
            s3_path = parts[1]
            video_mapping[video_id] = s3_path

In [ ]:
def calculate_actual_duration(clip_path):
    try:
        ffmpeg_command = f"ffprobe -i {clip_path} -show_entries format=duration -v quiet -of csv=p=0"
        duration = float(os.popen(ffmpeg_command).read().strip())
        return duration
    except Exception as e:
        print(f"Error calculating duration for {clip_path}: {e}")
        return None

# Step 2: Download and extract clips
# Enhanced Script for Downloading and Verifying Clips
for query in top_queries:
    video_id = query["video_id"]
    ground_truth = query["ground_truth"]
    start_time, end_time = ground_truth

    if video_id not in video_mapping:
        print(f"Video ID {video_id} not found in manifest.")
        continue

    # Download the full video
    s3_video_path = video_mapping[video_id]
    video_file = f"/content/{video_id}.mp4"
    download_command = f"aws s3 cp {s3_video_path} {video_file}"
    print(f"Downloading video: {video_file}")
    os.system(download_command)

    if not os.path.exists(video_file):
        print(f"Failed to download video: {video_file}")
        continue
    ##
    original_video = calculate_actual_duration(video_file)
    print(f"Original video length --> {original_video}")
    ##
    # Extract the relevant clip using FFmpeg
    clip_output_path = os.path.join(output_dir, f"{video_id}__{start_time}__{end_time}.mp4")
    #     the following command is faster but gave us problems with two clips
    #     ffmpeg_command = f"ffmpeg -i {video_file} -ss {start_time} -to {end_time} -c copy {clip_output_path}"
    ffmpeg_command = f"ffmpeg -i {video_file} -ss {start_time} -to {end_time} -c:v libx264 -preset fast -crf 23 -c:a aac {clip_output_path}"
    print(f"Extracting clip: {clip_output_path}")
    os.system(ffmpeg_command)

    if not os.path.exists(clip_output_path):
        print(f"Failed to extract clip: {clip_output_path}")
        continue

    # Verify clip duration
    actual_duration = calculate_actual_duration(clip_output_path)
    expected_duration = end_time - start_time
    if actual_duration is None:
        print(f"Duration could not be calculated for: {clip_output_path}")
        os.remove(clip_output_path)
    elif abs(actual_duration - expected_duration) > 0.1:  # Allow small tolerance
        print(f"Clip duration mismatch: Expected {expected_duration}s, Got {actual_duration}s")
        os.remove(clip_output_path)
    else:
        print(f"Successfully extracted clip: {clip_output_path}")

    # Optional: Remove the full video to save space
    if os.path.exists(video_file):
        os.remove(video_file)

### Zip the directory and download

In [ ]:
!zip -r /content/extracted_clips.zip /content/extracted_clips